# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.multioutput import MultiOutputClassifier
import pickle
from sklearn import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Miriam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Miriam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///ETL_disaster.db')
df_etl =pd.read_sql('SELECT * from ETL_disaster', engine)
X = df_etl['message']
Y = df_etl.drop(['id', 'message', 'genre', 'original'], axis = 1) #remove the columns without dummies

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
 pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [8]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_we

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

    # train classifier
pipeline.fit(X_train, y_train)

    # predict on test data


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
y_pred = pipeline.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('Attribute: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))


Attribute: related

             precision    recall  f1-score   support

          0       0.56      0.45      0.50      1507
          1       0.84      0.89      0.87      4998
          2       1.00      0.20      0.34        49

avg / total       0.78      0.79      0.78      6554

Attribute: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5421
          1       0.84      0.33      0.48      1133

avg / total       0.87      0.87      0.85      6554

Attribute: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

Attribute: aid_related

             precision    recall  f1-score   support

          0       0.68      0.91      0.78      3786
          1       0.77      0.42      0.54      2768

avg / total       0.72      0.70      0.68      6554

Attri

C:\Users\Miriam\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Attribute: aid_centers

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6490
          1       0.00      0.00      0.00        64

avg / total       0.98      0.99      0.99      6554

Attribute: other_infrastructure

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6260
          1       0.00      0.00      0.00       294

avg / total       0.91      0.95      0.93      6554

Attribute: weather_related

             precision    recall  f1-score   support

          0       0.80      0.97      0.88      4694
          1       0.85      0.40      0.54      1860

avg / total       0.82      0.81      0.78      6554

Attribute: floods

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6012
          1       0.87      0.17      0.28       542

avg / total       0.92      0.93      0.91      6554

Attribute: storm

             prec

### 6. Improve your model
Use grid search to find better parameters. 

In [31]:

parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_split': [2, 3, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [42]:
def new_build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier))
    ])

    parameters = {
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'clf__max_depth': [10, 20, 30],
        'clf__min_samples_leaf': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

    return cv

In [43]:

tuned_model = cv.fit(X_train, y_train)

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('Attribute: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
cv.best_params_

In [ ]:
def build_model():
    pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutPutClassifier(SVC()))
    ])

    parameters = { 
        'features__text_pipeline__vect__max_df': (1.0),
        'features__text_pipeline__tfidf__use_idf': (True),
        'clf__estimator_kernel': ['poly'],
        'clf__estimator_degree': [1, 2, 3],
        'clf__estimator_C': [1, 10, 100]
        )
    }

    cv1 = GridSearchCV(pipeline2, param_grid=parameters)

    return cv1

In [ ]:
np.random.seed(81)
tuned_model = cv1.fit(X_train, Y_train)
 # predict on test data
y_pred1 = cv1.predict(X_test)

    # display results
display_results(y_test, y_pred1)

In [ ]:
print '\nClasification report:\n', classification_report(y_test, y_pred1)
print '\nConfussion matrix:\n',confusion_matrix(y_test, y_pred1)

In [ ]:
cv1.best_params_

### 9. Export your model as a pickle file

In [ ]:
clf_model = pickle.dumps(clf)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.